# Fitting: Computing an NLL

We will be using Tensorflow's new eager mode, the new JIT static graph, and a classic API static graph to solve a different problem: fitting unbinned datasets. Like before, let's set up the data and then try a solution with Numpy:

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import math

np.random.seed(42)

dist = np.hstack([
    np.random.normal(loc=1, scale=2., size=500_000),
    np.random.normal(loc=1, scale=.5, size=500_000)
])

In [ ]:
import tensorflow as tf

In [ ]:
d_dist = tf.constant(dist, name='dist')

In [ ]:
def gaussian(x, μ, σ):
    return 1/tf.sqrt(2*np.pi*σ**2) * tf.math.exp(-(x-μ)**2/(2*σ**2))

def add(x, f_0, mean, sigma, sigma2):
    return f_0 * gaussian(x, mean, sigma) + (1 - f_0) * gaussian(x, mean, sigma2)

def make_nll(dist, f_0, mean, sigma, sigma2):
    return -tf.reduce_sum(tf.math.log(add(dist, f_0, mean, sigma, sigma2)))

In [ ]:
%%timeit
make_nll(d_dist, *np.random.rand(4))

This could be written as a decorator, `@tf.function`:

In [ ]:
nll = tf.function(make_nll)

In [ ]:
tf_f_0 = tf.Variable(0, dtype=tf.float64)
tf_mean = tf.Variable(0, dtype=tf.float64)
tf_sigma = tf.Variable(0, dtype=tf.float64)
tf_sigma2 = tf.Variable(0, dtype=tf.float64)

In [ ]:
%%timeit
r = np.random.rand(4)

tf_f_0.assign(r[0])
tf_mean.assign(r[1])
tf_sigma.assign(r[1])
tf_sigma2.assign(r[1])

nll(d_dist, tf_f_0, tf_mean, tf_sigma, tf_sigma2)

In [ ]:
import tensorflow.compat.v1 as tf

In [ ]:
tf.disable_eager_execution()

In [ ]:
def gaussian(x, μ, σ):
    return 1/tf.sqrt(2*np.pi*σ**2) * tf.math.exp(-(x-μ)**2/(2*σ**2))

def add(x, f_0, mean, sigma, sigma2):
    return f_0 * gaussian(x, mean, sigma) + (1 - f_0) * gaussian(x, mean, sigma2)

def make_nll(dist, f_0, mean, sigma, sigma2):
    return -tf.reduce_sum(tf.math.log(add(dist, f_0, mean, sigma, sigma2)))

In [ ]:
x = tf.constant(dist)
tf_f_0 = tf.placeholder(dtype=tf.float64)
tf_mean = tf.placeholder(dtype=tf.float64)
tf_sigma = tf.placeholder(dtype=tf.float64)
tf_sigma2 = tf.placeholder(dtype=tf.float64)

graph = make_nll(x, tf_f_0, tf_mean, tf_sigma, tf_sigma2)

In [ ]:
sess = tf.Session()

def nll(f_0, mean, sigma, sigma2): 
    loss_value = sess.run(graph,
                          feed_dict={tf_f_0:f_0,
                                     tf_mean:mean,
                                     tf_sigma:sigma,
                                     tf_sigma2:sigma2})
    return loss_value

In [ ]:
%%timeit
nll(*np.random.rand(4))

In [ ]:
sess.close()